# Task04 大语言模型

## 1 LLM概念梳理

**LLM的定义：**
1. 中文名：大语言模型或大型语言模型。
2. 包含数百亿（或更多）参数的语言模型。
3. 在数T量级 token 语料上通过多卡分布式集群进行预训练。

**LLM的能力：**
1. 涌现能力（量变引起了质变）：随着研究的不断深入、高质量数据的不断涌现和更高效的模型架构及训练框架的出现，LLM最终能具备通用人工智能的能力。
2. 上下文学习：大模型通过理解上下文并生成相应输出的方式来执行任务，而无需额外的训练或参数更新。
3. 指令遵循：在指令微调阶段混合多种指令来训练其泛化能力。
4. 逐步推理：通过采用思维链（CoT）推理策略，可以利用包含中间推理步骤的提示机制来解决这些任务，从而得出最终答案。

**LLM的特点：**
1. 多语言支持
2. 长文本处理：大部分采用了旋转位置编码作为位置编码，提升长度外推能力。
3. 拓展多模态：支持文字、图像双模态的模型。
4. 幻觉问题：LLM 根据 Prompt 杜撰生成虚假、错误信息。

## 2 训练一个LLM

### 2.1 Pretrain

- 预训练数据：大规模的预训练数据，根据由 OpenAI 提出的 Scaling Law：C ~ 6ND，其中 C 为计算量，N 为模型参数，D 为训练的 token 数，可以实验得出训练 token 数应该是模型参数的1.7倍。
- 算力资源：百亿级 LLM 需要 1024张 A100 训练一个多月，而十亿级 LLM 一般也需要 256张 A100 训练2~3天。
- 分布式训练框架：数据并行和模型并行。主流的分布式训练框架包括 Deepspeed、Megatron-LM、ColossalAI。
- 预训练语料：主要的开源预训练语料包括 CommonCrawl、C4、Github、Wikipedia 等
- 预训练数据处理：文档准备、语料过滤、语料去重等。

### 2.2 SFT

- SFT的主要目标：让模型从多种类型、多种风格的指令中获得泛化的指令遵循能力。
- 指令数据量：在单个任务上的训练样本为500~1000条，可以满足不错的微调效果。
- 指令数据集的覆盖范围：文本生成、开放域问答、头脑风暴、聊天、文本转写、文本总结、文本分类、特定域问答、文本抽取和其他。
- 指令数据集的结构：
```json
{
    "instruction":"即输入的用户指令",
    "input":"执行该指令可能需要的补充输入，没有则置空",
    "output":"即模型应该给出的回复"
}
```

### 2.3 RLHF

- RLHF简单描述：从更深层次令 LLM 和人类价值观对齐，令其达到安全、有用、无害的核心标准。
- 实现思路：引入强化学习的技术，通过实时的人类反馈令 LLM 能够给出更令人类满意的回复。
- RM（奖励模型）：训练的偏好数据往往是由人工标注的，模型难以拟合到正确的标量奖励。将大模型的不同回复进行排名，再将排名转化为奖励。
- PPO算法（近端策略优化算法）：
    1. 两个 SFT 之后的 LLM 使用指令数据集得到两个输出回复。
    2. 计算两个输出回复的 KL 散度。
    3. 计算两个模型的打分。
    4. 将 KL 散度和两个模型的打分均输入到奖励函数中得到损失值。
    5. 更新两个模型的参数。